In [1]:
pip install -q -U transformers accelerate peft bitsandbytes torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 112.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 135.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [4]:
import zipfile
import os

print("Extracting adapter...")
with zipfile.ZipFile('phi3-mini_bose_qa_model.zip', 'r') as zip_ref:
    zip_ref.extractall('./adapter')

print("✅ Adapter extracted to ./adapter")
print(f"Contents: {os.listdir('./adapter')}")

Extracting adapter...
✅ Adapter extracted to ./adapter
Contents: ['tokenizer_config.json', 'chat_template.jinja', 'checkpoint-200', 'checkpoint-100', 'tokenizer.model', 'tokenizer.json', 'checkpoint-300', 'adapter_config.json', 'special_tokens_map.json', 'adapter_model.safetensors', 'added_tokens.json', 'checkpoint-400', 'README.md', 'checkpoint-500']


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"
ADAPTER_PATH = "./adapter"
OUTPUT_PATH = "./phi-bose-merged"

print("\n" + "=" * 60)
print("Loading base Phi-3.5-mini model...")
print("=" * 60)

# Load base model in FP16 (lighter than FP32)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

print("✅ Base model loaded")


Loading base Phi-3.5-mini model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`torch_dtype` is deprecated! Use `dtype` instead!


...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

✅ Base model loaded


In [6]:
print("\n" + "=" * 60)
print("Loading LoRA adapter and merging...")
print("=" * 60)

# Load the adapter
model_with_adapter = PeftModel.from_pretrained(base_model, ADAPTER_PATH)

# Merge the adapter weights into the base model
merged_model = model_with_adapter.merge_and_unload()

print("✅ Adapter merged successfully!")


Loading LoRA adapter and merging...
✅ Adapter merged successfully!


In [7]:
print("\n" + "=" * 60)
print("Saving merged model...")
print("=" * 60)

# Save the merged model
merged_model.save_pretrained(OUTPUT_PATH)

# Load and save tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.save_pretrained(OUTPUT_PATH)

print(f"✅ Merged model saved to {OUTPUT_PATH}")


Saving merged model...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

✅ Merged model saved to ./phi-bose-merged


In [9]:
print("\n" + "=" * 60)
print("Testing merged model...")
print("=" * 60)

test_prompt = "### User: Can I use the speaker outdoor?. ### Assistant:"
inputs = tokenizer(test_prompt, return_tensors="pt").to(merged_model.device)

with torch.no_grad():
    outputs = merged_model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        use_cache=False
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"\nTest output:\n{response}\n")


Testing merged model...

Test output:
### User: Can I use the speaker outdoor?. ### Assistant: Yes, the DM8SE can be used outdoors and is weather-sealed.



In [ ]:
import shutil

# Create a zip file of the merged model
shutil.make_archive('phi3mini-bose-merged', 'zip', OUTPUT_PATH)

# the zip file was downloaded from the runtime files in colab